<a href="https://colab.research.google.com/github/nalinzip/SCC/blob/main/tid0358_scc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Skill Gap Recommender - With Top % Input from User

# Install dependencies in Colab if needed
# !pip install -q sentence-transformers ipywidgets scikit-learn seaborn

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from sentence_transformers import SentenceTransformer, util
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import seaborn as sns

# Load Excel file
xls = pd.ExcelFile("/content/sample_data/DataSourceBanpu.xlsx")
skill_df = xls.parse('Skill')
employee_df = xls.parse('EmployeeData')

# Clean and merge
df = skill_df.merge(employee_df, on='Name', how='left')
df.columns = df.columns.str.strip()
df['combined_text'] = df[['Job Title', 'Experience', 'Skill']].fillna('').agg(' '.join, axis=1).str.lower()

# Load pretrained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Auto Skill Tagging (rule-based)
known_skills = [
    "python", "excel", "analytics", "risk management", "energy market",
    "forecasting", "sql", "data analysis", "machine learning", "leadership",
    "strategy", "communication", "project management"
]

def extract_skills_from_text(text, skills=known_skills):
    text = text.lower()
    return [skill for skill in skills if skill in text]

df['AutoTagged_Skills_List'] = df['combined_text'].apply(lambda x: extract_skills_from_text(x))
df['AutoTagged_Skills'] = df['AutoTagged_Skills_List'].apply(lambda x: ", ".join(x))

# Profile templates
example_profiles = {
    'Future Capabilities Team': "Expertise in trading strategy, energy markets, decision making, global perspective, quantitative risk management, and data analysis including price forecasting.",
    'Energy Trading Expert': "Specializes in energy trading, market analysis, and risk management.",
    'Technology Transformation Leader': "Experienced in digital transformation, innovation, IT strategy, and technology implementation.",
    'Global Business Strategist': "Global experience with strategic planning, international business, and multicultural team management."
}

# Widgets
profile_textbox = widgets.Textarea(
    value=example_profiles['Future Capabilities Team'],
    description='Target Profile:',
    layout=widgets.Layout(width='100%', height='100px')
)

top_percent_slider = widgets.FloatSlider(
    value=30.0,
    min=10.0,
    max=100.0,
    step=5.0,
    description='Top %:',
    layout=widgets.Layout(width='50%')
)

run_button = widgets.Button(description="Analyze")
output = widgets.Output()

def analyze_team(b):
    with output:
        clear_output(wait=True)
        target_profile = profile_textbox.value.strip()
        print("Target profile:\n" + target_profile)

        employee_embeddings = model.encode(df['combined_text'].tolist(), convert_to_tensor=True)
        target_embedding = model.encode(target_profile, convert_to_tensor=True)
        similarity_scores = util.cos_sim(employee_embeddings, target_embedding).cpu().numpy().flatten()
        df['Similarity_Score'] = similarity_scores

        highlight_keywords = ['trading', 'strategy', 'energy', 'market', 'decision', 'leadership',
                              'strategic', 'global', 'international', 'risk', 'quantitative',
                              'forecasting', 'analytics', 'price']

        df['Key_Strengths_List'] = df['combined_text'].apply(lambda text: [kw for kw in highlight_keywords if kw in text])
        df['Key_Strengths'] = df['Key_Strengths_List'].apply(lambda x: ", ".join(x))

        X_embed = employee_embeddings.cpu().numpy()
        y_level = pd.qcut(df['Similarity_Score'], q=3, labels=['Beginner', 'Intermediate', 'Expert'])
        le = LabelEncoder()
        y_encoded = le.fit_transform(y_level)

        X_train, X_test, y_train, y_test = train_test_split(X_embed, y_encoded, test_size=0.3, random_state=42)
        clf = LogisticRegression(max_iter=1000, random_state=42)
        clf.fit(X_train, y_train)

        df['Expertise_Level'] = le.inverse_transform(clf.predict(X_embed))
        df['Expertise_Level'] = pd.qcut(df['Similarity_Score'], q=3, labels=['Beginner', 'Intermediate', 'Expert'])

        percent_value = top_percent_slider.value / 100.0
        threshold = df['Similarity_Score'].quantile(1 - percent_value)
        selected_df = df[df['Similarity_Score'] >= threshold].sort_values(by='Similarity_Score', ascending=False)
        not_selected_df = df[df['Similarity_Score'] < threshold].sort_values(by='Similarity_Score', ascending=False)

        print(f"\nTop {top_percent_slider.value:.0f}% Selected Employees:")
        display(selected_df[['Name', 'Job Title', 'Similarity_Score', 'Key_Strengths', 'AutoTagged_Skills']])

        print("\nRemaining Employees (Not Selected):")
        display(not_selected_df[['Name', 'Job Title', 'Similarity_Score']].head(10))

        print("\nExpertise Levels:")
        display(df[['Name', 'Job Title', 'Similarity_Score', 'Expertise_Level']].sort_values(by='Similarity_Score', ascending=False).head(10))

        all_strengths = sum(selected_df['Key_Strengths_List'], [])
        if all_strengths:
            strength_counts = pd.Series(all_strengths).value_counts(normalize=True).sort_values(ascending=False) * 100
            print("\nKey Strength Distribution in Selected Team (%):")
            display(strength_counts.to_frame("Percentage"))

            strength_counts.plot(kind='pie', figsize=(6, 6), autopct='%1.1f%%', startangle=90)
            plt.title('Team Strength Composition')
            plt.ylabel('')
            plt.tight_layout()
            plt.show()

        print("\nML Insight: Clustering with PCA")
        pca = PCA(n_components=2)
        reduced = pca.fit_transform(X_embed)
        kmeans = KMeans(n_clusters=3, random_state=42)
        labels = kmeans.fit_predict(reduced)
        df['Cluster'] = labels

        plt.figure(figsize=(7, 5))
        sns.scatterplot(x=reduced[:, 0], y=reduced[:, 1], hue=labels, palette='Set2', s=60)
        plt.title('PCA Clustering of Employee Profiles')
        plt.xlabel('PCA 1')
        plt.ylabel('PCA 2')
        plt.legend(title='Cluster')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        cluster_centroids = pd.DataFrame(X_embed).groupby(df['Cluster']).mean()
        centroid_similarities = cosine_similarity(cluster_centroids.values, target_embedding.cpu().numpy().reshape(1, -1)).flatten()
        df['Cluster_Similarity'] = df['Cluster'].map(dict(zip(cluster_centroids.index, centroid_similarities)))
        cluster_info = df.groupby('Cluster')[['Cluster_Similarity']].first().sort_values(by='Cluster_Similarity', ascending=False)
        print("\nIndirectly Similar Clusters to Target Profile:")
        display(cluster_info)

        print("\nML Insight: Semantic Space via t-SNE")
        tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
        tsne_result = tsne.fit_transform(X_embed)
        plt.figure(figsize=(7, 5))
        sns.scatterplot(x=tsne_result[:, 0], y=tsne_result[:, 1], hue=labels, palette='Set2', s=60)
        plt.title('t-SNE Visualization of Semantic Profile Space')
        plt.xlabel('t-SNE 1')
        plt.ylabel('t-SNE 2')
        plt.legend(title='Cluster')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        print("\nSkill Area Gap Analysis:")
        df['Skill_List'] = df['Skill'].fillna('').str.lower().str.split(",\s*")

        skill_gap_summary = defaultdict(list)
        all_skills = [skill for skills in df['Skill_List'] if isinstance(skills, list) for skill in skills]

        for skill in sorted(set(all_skills)):
            skill_group = df[df['Skill_List'].apply(lambda skills: skill in skills if isinstance(skills, list) else False)]
            if not skill_group.empty:
                avg_score = skill_group['Similarity_Score'].mean()
                num_experts = (skill_group['Expertise_Level'] == 'Expert').sum()
                skill_gap_summary['Skill'].append(skill)
                skill_gap_summary['Avg_Similarity'].append(round(avg_score, 3))
                skill_gap_summary['Expert_Count'].append(num_experts)
                skill_gap_summary['Total_Employees'].append(len(skill_group))

        skill_gap_df = pd.DataFrame(skill_gap_summary).sort_values(by='Avg_Similarity')
        display(skill_gap_df.head(10))

# Show UI
print(" You can edit the profile and select top % below:")
display(profile_textbox)
display(top_percent_slider)
display(run_button, output)
run_button.on_click(analyze_team)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

 You can edit the profile and select top % below:


Textarea(value='Expertise in trading strategy, energy markets, decision making, global perspective, quantitati…

FloatSlider(value=30.0, description='Top %:', layout=Layout(width='50%'), min=10.0, step=5.0)

Button(description='Analyze', style=ButtonStyle())

Output()